# Zero & Few Shot Prompts

In [ ]:
# !pip install cohere

In [87]:
from dotenv import load_dotenv
import os

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')


In [75]:
from huggingface_hub import InferenceClient

model_id = 'tiiuae/falcon-7b-instruct'

# model_id = 'google/flan-t5-xxl'

client = InferenceClient(model=model_id)

## Try Zero-Shot

In [76]:
prompt = """
Return a list of named entities and their types in the text.

Text: john travelled with his dog duffy from texas to Turkey and met with his friend abdul
"""

client.text_generation(prompt, max_new_tokens=75)

'<pre><code>[John] [Person] [Animal] [Adul] [Friend]\n</code></pre>'

## Instruct the output to be in JSON format

In [77]:
prompt = """
Return a list of named entities and their types in the text.

Format response as JSON

Text: john travelled with his dog duffy from texas to Turkey and met with his friend abdul
Answer:
"""

client.text_generation(prompt, max_new_tokens=100)

'\nHere is the list of named entities and their types in the text:\n\nPerson: john\nName: John\nType: Person\n\nAnimal: dog\nName: dog\nType: Animal\n\nPlace: texas\nName: Texas\nType: Place\n\nPerson: abdul\nName: Abdul\nType: Person\n\nPlace: Turkey\nName: Turkey\nType: Place\n\nThe response should be in JSON format and should look like this:\n\n{\n  "'

## 1-Shot

We want the JSON to be in this format: [{entity: type}, {entity : type}....]

All locattions should have a type = place

In [80]:
prompt = """
Return a list of named entities and their types in the text.

Format response as JSON shown in answer

Text: Anil lives in Paris, the most beautiful city in France
Answer: [{'entity': 'Anil', 'type': 'person'}, {'entity': 'Paris', 'type' : 'place'}, {'entity': 'France', 'type' : 'place'}]


Text: john travelled with his dog duffy from texas to Turkey and met with his friend abdul
Answer: 
"""
# Use the stop_sequences parameter to stop the generation after a "]" is output by model
# THERE IS ERROR IN HF DOCUMENTATION - it shows the parameter name as stop instead of stop_sequences
client.text_generation(prompt, max_new_tokens=75,stop_sequences=["]"])

"Answer: [{'entity': 'john', 'type': 'person'}, {'entity': 'duffy', 'type': 'animal'}, {'entity': 'abdul', 'type': 'person'}]\n\n\nText: The most beautiful city in the world is Paris\nAnswer: [{'entity"

## 2-Shot

In [79]:
prompt = """
Return a list of named entities and their types in the text.

Format response as JSON. Stop generating once valid JSON is created.

Text: Anil lives in Paris the most beautiful city in France
Answer: [{'entity': 'Anil', 'type': 'person'}, {'entity': 'Paris', 'type' : 'place'}, {'entity': 'France', 'type' : 'place'}]

Text: Jim lives in texas and he owns a cat named Alexa
Answer: [{'entity': 'Jim', 'type':'person'}, {'entity': 'texas', 'type': 'place'}, {'entity': 'alexa', 'type': 'cat'}]

Text: john travelled with his dog duffy from Texas to Turkey and met with his friend abdul
Answer:
"""

client.text_generation(prompt, max_new_tokens=75, stop_sequences=["]"])

"[{'entity': 'john', 'type': 'person'}, {'entity': 'Turkey', 'type': 'place'}, {'entity': 'Abdul', 'type': 'person'}]\n\n<p>Here is the code to generate the JSON response:</p>\n\n<pre><code>import json\n"

## Few -Shots

In [81]:
prompt = """
Return a list of named entities and their types in the text.

Return a list of named entities and their types in the text.

Format response as JSON. Stop generating once valid JSON is created.

Text: Anil lives in Paris the most beautiful city in France
Answer: [{'entity': 'Anil', 'type': 'person'}, {'entity': 'Paris', 'type' : 'place'}, {'entity': 'France', 'type' : 'place'}]

Text: Jim lives in texas and he owns a cat named Alexa
Answer: [{'entity': 'Jim', 'type':'person'}, {'entity': 'texas', 'type': 'place'}, {'entity': 'alexa', 'type': 'cat'}]

Text: Mary loves her hometown in Calfiornia, where she lives with her husband Paul
Answer: [{'entity': 'Mary', 'type':'person'}, {'entity': 'Calfiornia', 'type': 'state'},{'entity': 'Paul', 'type':'person'}]

Text: Jim is friends with Brad  who lives in LA in California
Answer: [{'entity': 'Jim', 'type':'person'}, {'entity': 'LA', 'type': 'city'}, {'entity': 'Calfiornia', 'type': 'state'}{'entity': 'Brad', 'type':'person'}]


Text: john travelled with his dog duffy from New Jersey to Turkey and met with his friend abdul
Answer:
"""

client.text_generation(prompt, max_new_tokens=75, stop_sequences=["]"])

"Answer: [{'entity': 'john', 'type':'person'}, {'entity': 'duffy', 'type': 'animal'}, {'entity': 'Abdul', 'type': 'person'}]\n\n\nHere is the code to achieve the desired result:\n\n```\nimport requests\n\ndef get_entities"

In [ ]:
text = "using the LLM you can build intelligent apps"


prompt = f"what category out of these ['news', 'health', 'finance'], does this article belong to : "

client.text_generation(prompt)

## Limitation with few-shot technique

### Few shot may not produce the desired result for reasoning/mathematics/logical problems

Try out this prompt with 

* Chat GPT 3.5

* Chere Command (use playground)

* Google Bard   (uses Gemini AI / LaMDA model)

You will realize that it works for Bard and it doesn't work for Chat GPT 3.5 or Cohere Command!!!

Do your research to figure out why is that the case. Share your results on the Q&A forum.

## Create a cohere client

Build a few shot prompt in an incremental fashion.

In [88]:
import cohere
co = cohere.Client(COHERE_API_KEY)

### 0 Shot prompt

In [114]:
prompt = """
Palindrome is a word that reads the same forward and backward. 

Your task is to identify palindrome words in the given sentence. 

sentence: {}
palindromes:

generate response in this format: palindrome1, palindrome2, ...
"""

input_text = "Civic authorities drove the kayak across the level lake."

response = co.generate(
  model='command',
  prompt=prompt.format(input_text),
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output

' Civic, kayak, level, lake'

### Few shots (Provide palindrome words as examples)

In [113]:
prompt = """
Palindrome is a word that reads the same forward and backward. 

Here are examples of palindrome words 

examples: radar, rotor, reviver, madam

Your task is to identify palindrome words in the given sentence. 

sentence: {}
palindromes:

generate response in this format: palindrome1, palindrome2, ...
"""

input_text = "Civic authorities drove the kayak across the level lake."

response = co.generate(
  model='command',
  prompt=prompt.format(input_text),
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output

' civic authorities, lake level, drove across, kayak across'

### Few shots (Provide sentences with palindromes as examples)

In [ ]:
prompt = """
Palindrome is a word that reads the same forward and backward. 

Your task is to identify palindrome words in the given sentence. 
Here are some examples.

sentence: Noon is a time when racecar enthusiasts gather.
palindromes: noon, racecar

sentence: Deed records show a radar signal in the evening.
palindromes: deed, radar

sentence: Reviver potions can revive a tired system
palindromes: reviver

sentence: rotors rotate, making a subtle rotor sound
palindromes: rotor


sentence: {}
palindromes:

generate response in this format: palindrome1, palindrome2, ...
"""

input_text = "Civic authorities drove the kayak across the level lake."

response = co.generate(
  model='command-light',
  prompt=prompt.format(input_text),
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output

In [ ]:
prompt = """
Palindrome is a word that reads the same forward and backward. 

Your task is to identify palindrome words in the given sentence. 

text: {}
palindromes:

generate response in this format: palindrome1, palindrome2, ...
"""

input_text = "Civic authorities drove the kayak across the level lake."

response = co.generate(
  model='command-light',
  prompt=prompt.format(input_text),
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output

### WAIT

In [100]:
prompt = """
use examples below to extract all palindrome words from the given text. Just give the words

example: Noon is a time when racecar enthusiasts gather.
answer: noon, racecar

example: Deed records show a radar signal in the evening.
answer: deed

example: Madam Anna made a beautiful mural for the wall
answer: madam


text: Civic authorities drove the kayak across the level lake
answer:
"""

response = co.generate(
  model='command-light',
  prompt=prompt,
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output


Palindrome is a word that reads the same forward and backward. 

There may be more than one palindrome words in a sentence, so list all words.

Use examples below for reference to identify all palindrome in given text.

example: Noon is a time when racecar enthusiasts gather.
answer: noon, racecar

example: Deed records show a radar signal in the evening.
answer: deed, radar

example: Madam Anna made a beautiful mural for the wall
answer: madam

example: Reviver potions can revive a tired system
answer: reviver

example: rotors rotate, making a subtle rotor sound
example: rotor

text: Civic authorities drove the kayak across the lake
answer:

' lake, kayak, across, authorities, drive'

In [79]:
from huggingface_hub import InferenceClient

hugging_face_model_ids = [
    'tiiuae/falcon-7b-instruct',
    'mistralai/Mixtral-8x7B-Instruct-v0.1',
    'openlm-research/open_llama_3b_v2',
    'google/flan-t5-xxl',
    'EleutherAI/gpt-neox-20b'
]


## Classification with few shots

In [80]:
llm = InferenceClient(model=hugging_face_model_ids[4])

In [81]:
prompt = """
extract palindrome words from the given text

text: Noon is a time when racecar enthusiasts gather.
answer: noon, racecar

text: Deed records show a radar signal in the evening.
answer: deed

text: Madam Anna made a civic mural on the wall
answer: madam 


text: Civic duty drove the kayak across the level lake
answer:
"""

llm.text_generation(prompt)

'\ntext: The man was a good citizen.\nanswer: man\n\ntext: The man'

## Zero shot model - example

In [61]:
llm = InferenceClient(model=hugging_face_model_ids[1])

In [68]:
prompt = """
[INST]extract all palindrome words from the given text. Just give the words[/INST]

text: Noon is a time when racecar enthusiasts gather.
answer: 
"""

llm.text_generation(prompt)

'\n* Noon\n* racecar\n\n(Note: case is ignored, so "No'

## Limitation of few shots

Example demonstrates the limitation of few-shots technique with math/reasoning tasks

In [47]:
llm = InferenceClient(model=hugging_face_model_ids[3])

In [142]:
prompt = """
generate a yes or a no as response. 

the odd numbers in this list [1,2,1,9,1] add up to an odd number 

"""

response = co.generate(
  model='command',
  prompt=prompt.format(input_text),
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output

' Yes.'

In [144]:
prompt ="""
refer to examples and answer generate a yes or a no as response. 

examples: the odd numbers in this list [1,3,4,5,2] adds up to an odd number//yes

examples: the odd numbers in this list [1,2,4,5,6] adds up to an odd number//no

examples: the odd numbers in this list [1,2,4,7,9] adds up to an odd number//yes 

examples: the odd numbers in this list [1,3,4,7,9] adds up to an odd number//no 

examples: the odd numbers in this list [1,2,1,9,2] adds up to an odd number//

"""

response = co.generate(
  model='command',
  prompt=prompt.format(input_text),
  max_tokens=50,
  temperature=0.3,
  k=0,
  stop_sequences=[],
  return_likelihoods='NONE')

output = response.generations[0].text

output

CohereAPIError: You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions